In [59]:

from scipy.stats import zscore

import datetime

import pandas as pd
import numpy as np

import datapungi_fed as dpf
from bcb import sgs
import yfinance as yf
import investpy as inv
import talib
from talib import RSI, ROC, STOCH, STOCHRSI, WILLR, ATR, MACD, CDLKICKINGBYLENGTH, CDLENGULFING

import bs4 as bs
import pickle
import requests
import json

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
with open('../fred_token.json') as arquivo:
  fed_credencial = json.load(arquivo)

In [7]:
def risco_brasil(): 
    resp = requests.get('http://www.ipeadata.gov.br/ExibeSerie.aspx?serid=40940&module=M')
    soup = bs.BeautifulSoup(resp.text)
    table = soup.find('table', { 'class': 'dxgvTable'})
    embi_risco_brasil =[]
    data = []
    df_risco_brasil = pd.DataFrame()
    for row in table.findAll('tr')[3:]:

        date = row.findAll('td')[0].text
        embi = row.findAll('td')[1].text
        data.append(date)
        embi_risco_brasil.append(embi)

    df_risco_brasil['data'] = data
    df_risco_brasil['Risco_Brasil'] = embi_risco_brasil
    
    df_risco_brasil['data'] = pd.to_datetime(df_risco_brasil['data'], format = '%d/%m/%Y')
    
    df_risco_brasil = df_risco_brasil.sort_values('data')
    df_risco_brasil.set_index("data", inplace = True)
    
    
    df_risco_brasil.to_csv('risco_brasil_{}.csv'.format(pd.to_datetime('today').date()))
    
    return df_risco_brasil

In [8]:
def consulta_bcb(nome, codigo, start='2000-01-01'):
    nome_= sgs.get((nome, codigo), start=start)
    
    nome_.index.name= 'data'
    return nome_

In [114]:
def mensal_diario_rate(serie_mensal):
    serie_day= serie_mensal.asfreq('B', method = 'pad' )
    daily = serie_day[serie_day.columns[0]].apply( lambda x:  round((1 + x) ** (1/22) - 1,3) ).to_frame()
    nome= daily.columns[0]
    daily.rename({nome: nome[:-6]+'diario' }, axis= 'columns', inplace = True)
    
    return daily

def mensal_diario_rate_nome(serie_mensal, name):
    serie_day= serie_mensal.asfreq('B', method = 'pad' )
    daily = serie_day.apply( lambda x:  round((1 + x) ** (1/22) - 1,3) ).to_frame()
    nome= daily.columns[0]
    daily.rename({nome: name }, axis= 'columns', inplace = True)
    
    
    return daily

In [147]:
def cenversao_simples_mensal_diario_B(serie_mensal):
    serie_day= serie_mensal.asfreq('B', method = 'pad' )
    daily = serie_day[serie_day.columns[0]].apply( lambda x:  round( x/22  ,3) ).to_frame()
    nome= daily.columns[0]
    daily.rename({nome: nome+'_diario' }, axis= 'columns', inplace = True)
    
    return daily

def conversao_simples_mensal_diario_B_nome(serie_diaria, name):
    serie_day= serie_diaria.asfreq('B', method = 'pad' )
    daily = serie_day.apply( lambda x:  round( x/22  ,3) ).to_frame()
    nome= daily.columns[0]
    daily.rename({nome: name }, axis= 'columns', inplace = True)
    
    return daily

In [11]:
def indicadores_tecnicos(stock):
    acao = yf.download(stock, start = '2000-01-01', end = datetime.datetime.today() - datetime.timedelta(days=1) )
    price = acao.iloc[::-1]
    price = price.dropna()
    close = price.Close.values
    open = price.Open.values
    high = price.High.values
    low = price.Low.values
    volume = price.Volume.values
    
    # RSI
    rsi = RSI(close, timeperiod=14)
    
    #ROC
    roc = ROC(close, timeperiod=10)
    
    #Williams %R moves between zero and -100.above -20 is overbought.-80 is oversold.
    willr = WILLR(high, low, close, timeperiod=14)
    
    data = { 'rsi': rsi,
            'roc': roc,
            'willr': willr
            
        
    }
    df = pd.DataFrame(data)
    
    return df

    

## Americano



#!pip install datapungi_fed

Onde pegar o token: 
https://fredaccount.stlouisfed.org/apikey

**O que Falta?**

- converter a inflacao_ameriana para REAL pois está em dolar,

- Juntar tudo no df com as info do Brasil
-df_dados_americanos_daily o index para 'data' ao inves de date

In [12]:
token= fed_credencial['token']

In [13]:

data = dpf.data(token)

In [14]:
#Producao Industrial Industrial Production Index - MENSAL - economic indicator that measures real output for all facilities located in the United States
industry_production = data.series('INDPRO')


#Taxa de Desemprego = MENSAL
desemprego_americano = data.series('UNRATE')

#Em Dolar The Consumer Price Index for All Urban Consumers (CPIAUCSL) is a price index of a basket of goods and services paid by urban consumers
inflacao_americana = data.series('CPIAUCSL')

# Tx de Juros americano- mês
tx_juros_americano = data.series('FEDFUNDS')

In [15]:
aux1 =pd.merge(industry_production, desemprego_americano, on="date", how='left')
aux2 = pd.merge(aux1, inflacao_americana, on="date", how='left' )
df_dados_americanos = pd.merge(aux2, tx_juros_americano, on="date", how='left' )

df_dados_americanos.rename({'INDPRO': 'ind_prod_americana', 'UNRATE': 'desemprego_americano', 'CPIAUCSL': 'consumer_priceUSD', 'FEDFUNDS': 'tx_juros_americano'}, axis= 'columns', inplace = True )

In [56]:
#df_dados_americanos.to_csv('../data/external/df_dados_americanos_monthly.csv')


In [62]:
# Graficos
aux = df_dados_americanos.loc['2000':]

px.bar(data_frame=aux.consumer_priceUSD.pct_change()*100, x=aux.index, y='consumer_priceUSD', labels={'consumer_priceUSD':'Inflação americada (%)'})




In [74]:
info_american_from2000= df_dados_americanos.loc['2000':]
info_american_from2000

,ind_prod_americana,desemprego_americano,consumer_priceUSD,tx_juros_americano
date,,,,
2000-01-01,91.7942,4.0,169.300,5.45
2000-02-01,92.1272,4.1,170.000,5.73
2000-03-01,92.4733,4.0,171.000,5.85
2000-04-01,93.0619,3.8,170.900,6.02
2000-05-01,93.3427,4.0,171.200,6.27
...,...,...,...,...
2021-10-01,101.2169,4.6,276.590,0.08
2021-11-01,102.0298,4.2,278.524,0.08
2021-12-01,101.6188,3.9,280.126,0.08


In [118]:
# Tx de juros - equivalencia financeira - a partir de 2000
tx_juros_americano_day = mensal_diario_rate_nome(info_american_from2000['tx_juros_americano'], 'tx_juros_americano_day')


In [149]:
# Conversão mensal para diário - conversão simple (/business day) - somente a partir de 2000
industry_production_daily = conversao_simples_mensal_diario_B_nome(info_american_from2000['ind_prod_americana'], 'industry_production_daily')
desemprego_americano_daily = conversao_simples_mensal_diario_B_nome(info_american_from2000['desemprego_americano'], 'desemprego_americano_day')
inflacao_americana_daily_USD = conversao_simples_mensal_diario_B_nome(info_american_from2000['consumer_priceUSD'], 'inflacao_americana_daily_USD')



In [150]:
aux1 =pd.merge(tx_juros_americano_day, industry_production_daily, on="date", how='left')
aux2 = pd.merge(aux1, desemprego_americano_daily, on="date", how='left' )
df_dados_americanos_daily = pd.merge(aux2, inflacao_americana_daily_USD, on="date", how='left' )
df_dados_americanos_daily

,tx_juros_americano_day,industry_production_daily,desemprego_americano_day,inflacao_americana_daily_USD
date,,,,
2000-01-03,0.088,4.172,0.182,7.695
2000-01-04,0.088,4.172,0.182,7.695
2000-01-05,0.088,4.172,0.182,7.695
2000-01-06,0.088,4.172,0.182,7.695
2000-01-07,0.088,4.172,0.182,7.695
...,...,...,...,...
2022-01-26,0.004,4.683,0.182,12.815
2022-01-27,0.004,4.683,0.182,12.815
2022-01-28,0.004,4.683,0.182,12.815


## BRASIL

***O que falta e apontamentos?***
- organização geral,
- Separar os dados para análise tecnica e fundamentalista
- muita coisa ...
- Dividendos: tratar como 'categorico'? Como identificar se próximo ao dia do pagamento de dividendos, houve oscilação no preço ou no volume de negoociações???

### Coletando as informações - para o df base

#### Yfinance

In [18]:
#Cotações
cotacoes = yf.download(['USDBRL=X', '^BVSP','ABEV3.SA' ], start = '2000-01-01', end = datetime.datetime.today() - datetime.timedelta(days=1) )['Adj Close']
cotacoes.rename({'ABEV3.SA': 'AMBEV', 'USDBRL=X': 'DOLAR', '^BVSP':'IBOVESPA'},axis= 'columns', inplace= True)
cotacoes.index.name = 'data'

[*********************100%***********************]  3 of 3 completed


In [19]:
cotacoes.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5788 entries, 2000-01-03 to 2022-03-30
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   AMBEV     5586 non-null   float64
 1   DOLAR     4336 non-null   float64
 2   IBOVESPA  5502 non-null   float64
dtypes: float64(3)
memory usage: 180.9 KB


In [20]:
# Instanciando o Ticker para pegar informações mais específicas da empresa
ticker = yf.Ticker('ABEV3.SA')

In [21]:
dividends = ticker.dividends.to_frame()
dividends.index.name= 'data'
dividends = dividends.asfreq('B', fill_value = 0)


#### scraping ipea

In [22]:
# Risco Brasil
risco_brasil = risco_brasil()


#### BANCO CENTRAL

In [23]:
#DIARIA Consulta Banco Central
cdi = consulta_bcb('cdi',12)
cdi

,cdi
data,
1999-12-31,0.068485
2000-01-03,0.068318
2000-01-04,0.068218
2000-01-05,0.068184
2000-01-06,0.068218
...,...
2022-03-23,0.043739
2022-03-24,0.043739
2022-03-25,0.043739


#### Pré juntando até aqui

In [24]:
junta_cdi = pd.merge(cotacoes, cdi, on= 'data', how= 'left')
junta_risco = pd.merge(junta_cdi, risco_brasil, on= 'data', how= 'left')
df_diaria = pd.merge(junta_risco, dividends,on= 'data', how= 'left' )
df_diaria.rename({'valor': 'CDI'}, axis= 'columns', inplace= True)

#Dividendos = se não tem valor é p.q não teve 
df_diaria.Dividends.fillna(0, inplace = True)

df_diaria.tail()

,AMBEV,DOLAR,IBOVESPA,cdi,Risco_Brasil,Dividends
data,,,,,,
2022-03-24,14.20,4.8242,119053.0,0.043739,313,0.0
2022-03-25,14.32,4.8263,119081.0,0.043739,301,0.0
2022-03-28,14.74,4.7369,118738.0,0.043739,301,0.0
2022-03-29,15.25,4.7641,120014.0,0.043739,300,0.0
2022-03-30,NaN,4.7562,NaN,NaN,NaN,0.0


In [25]:
# MENSAL - CONSULTA Banco Central
ipca_mensal = consulta_bcb('ipca_mensal',433)
ipca_alimentos_bebidas_mensal = consulta_bcb('ipca_alimentos_bebidas',1635)
igpm_mensal = consulta_bcb('igpm',189)
selic_meta_mensal = consulta_bcb('selic_meta_mensal',432)
selic_overnight_mensal = consulta_bcb('selic_overnight_mensal',4189)
taxa_desemprego_pnad_mensal = consulta_bcb('taxa_desemprego_pnad_mensal', 24369)
indice_volume_vendas_varejo_setor_mensal = consulta_bcb('indice_volume_vendas_varejo_setor_mensal',1496)

indice_producao_bens_consumo_mensal = consulta_bcb('indice_producao_bens_consumo_mensal',21867)

In [103]:
ipca_mensal

,ipca_mensal
data,
2000-01-01,0.62
2000-02-01,0.13
2000-03-01,0.22
2000-04-01,0.42
2000-05-01,0.01
...,...
2021-10-01,1.25
2021-11-01,0.95
2021-12-01,0.73


#### CONVERSAO MENSAL- DIARIO

In [26]:
##DIARIO - CONVERSAO DO MENSAL

# Txa juros, selic: Tem que fazer a equivalencia financeira
selic_meta_diario = mensal_diario_rate(selic_meta_mensal)
selic_overnight_diario = mensal_diario_rate(selic_overnight_mensal)
ipca_diario = mensal_diario_rate(ipca_mensal)


# Conversão simples valor mensal / 22 ( médias business day)
igpm_diario = cenversao_simples_mensal_diario_B(igpm_mensal)
taxa_desemprego_pnad_diario = cenversao_simples_mensal_diario_B(taxa_desemprego_pnad_mensal)
indice_volume_vendas_varejo_setor_diario = cenversao_simples_mensal_diario_B(indice_volume_vendas_varejo_setor_mensal)

#renomear
indice_volume_vendas_varejo_setor_diario.rename({'indice_volume_vendas_varejo_setor_mensal_diario': 'volume_vendas_varejo_setor_diario' }, axis= 'columns', inplace = True)


indice_producao_bens_consumo_diario = cenversao_simples_mensal_diario_B(indice_producao_bens_consumo_mensal)

#renomear
indice_producao_bens_consumo_diario.rename({'indice_producao_bens_consumo_mensal_diario': 'producao_bens_consumo_diario' }, axis= 'columns', inplace = True)


In [27]:
indice_volume_vendas_varejo_setor_diario.rename({'indice_volume_vendas_varejo_setor_mensal_diario': 'volume_vendas_varejo_setor_diario' }, axis= 'columns', inplace = True)


#### Juntando as info no DF

In [28]:
ax = pd.merge(selic_meta_diario, selic_overnight_diario, on= 'data', how= 'left' )
ax1  = pd.merge(ax,taxa_desemprego_pnad_diario , on= 'data', how= 'left' )
ax2  = pd.merge(ax1,indice_volume_vendas_varejo_setor_diario , on= 'data', how= 'left' )
ax3  = pd.merge(ax2,ipca_diario  , on= 'data', how= 'left' )
ax4  = pd.merge(ax3,igpm_diario  , on= 'data', how= 'left' )
ax5  = pd.merge(ax4,indice_volume_vendas_varejo_setor_diario  , on= 'data', how= 'left' )
ax6  = pd.merge(ax5,indice_producao_bens_consumo_diario , on= 'data', how= 'left' )

df_diaria_final = pd.merge(df_diaria, ax6, on='data', how='left')

In [151]:
df_diaria_final 

,AMBEV,DOLAR,IBOVESPA,cdi,Risco_Brasil,Dividends,selic_meta_diario,selic_overnight_diario,taxa_desemprego_pnad_mensal_diario,volume_vendas_varejo_setor_diario_x,ipca_diario,igpm_diario,volume_vendas_varejo_setor_diario_y,producao_bens_consumo_diario
data,,,,,,,,,,,,,,
2000-01-03,NaN,NaN,16930.0,0.068318,626,0.0,0.146,0.146,NaN,2.468,0.022,0.056,2.468,NaN
2000-01-04,NaN,NaN,15851.0,0.068218,675,0.0,0.146,0.146,NaN,2.468,0.022,0.056,2.468,NaN
2000-01-05,0.321626,NaN,16245.0,0.068184,677,0.0,0.146,0.146,NaN,2.468,0.022,0.056,2.468,NaN
2000-01-06,0.305323,NaN,16107.0,0.068218,688,0.0,0.146,0.146,NaN,2.468,0.022,0.056,2.468,NaN
2000-01-07,0.305323,NaN,16309.0,0.068218,677,0.0,0.146,0.146,NaN,2.468,0.022,0.056,2.468,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-24,14.200000,4.8242,119053.0,0.043739,313,0.0,0.123,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-03-25,14.320000,4.8263,119081.0,0.043739,301,0.0,0.123,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-03-28,14.740000,4.7369,118738.0,0.043739,301,0.0,0.123,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
df_diaria_final.columns

Index(['AMBEV', 'DOLAR', 'IBOVESPA', 'cdi', 'Risco_Brasil', 'Dividends',
       'selic_meta_diario', 'selic_overnight_diario',
       'taxa_desemprego_pnad_mensal_diario',
       'volume_vendas_varejo_setor_diario_x', 'ipca_diario', 'igpm_diario',
       'volume_vendas_varejo_setor_diario_y', 'producao_bens_consumo_diario'],
      dtype='object')

#### Indicadores técnicos
https://blog.quantinsti.com/install-ta-lib-python/

In [31]:
talib.get_functions()

['HT_DCPERIOD',
 'HT_DCPHASE',
 'HT_PHASOR',
 'HT_SINE',
 'HT_TRENDMODE',
 'ADD',
 'DIV',
 'MAX',
 'MAXINDEX',
 'MIN',
 'MININDEX',
 'MINMAX',
 'MINMAXINDEX',
 'MULT',
 'SUB',
 'SUM',
 'ACOS',
 'ASIN',
 'ATAN',
 'CEIL',
 'COS',
 'COSH',
 'EXP',
 'FLOOR',
 'LN',
 'LOG10',
 'SIN',
 'SINH',
 'SQRT',
 'TAN',
 'TANH',
 'ADX',
 'ADXR',
 'APO',
 'AROON',
 'AROONOSC',
 'BOP',
 'CCI',
 'CMO',
 'DX',
 'MACD',
 'MACDEXT',
 'MACDFIX',
 'MFI',
 'MINUS_DI',
 'MINUS_DM',
 'MOM',
 'PLUS_DI',
 'PLUS_DM',
 'PPO',
 'ROC',
 'ROCP',
 'ROCR',
 'ROCR100',
 'RSI',
 'STOCH',
 'STOCHF',
 'STOCHRSI',
 'TRIX',
 'ULTOSC',
 'WILLR',
 'BBANDS',
 'DEMA',
 'EMA',
 'HT_TRENDLINE',
 'KAMA',
 'MA',
 'MAMA',
 'MAVP',
 'MIDPOINT',
 'MIDPRICE',
 'SAR',
 'SAREXT',
 'SMA',
 'T3',
 'TEMA',
 'TRIMA',
 'WMA',
 'CDL2CROWS',
 'CDL3BLACKCROWS',
 'CDL3INSIDE',
 'CDL3LINESTRIKE',
 'CDL3OUTSIDE',
 'CDL3STARSINSOUTH',
 'CDL3WHITESOLDIERS',
 'CDLABANDONEDBABY',
 'CDLADVANCEBLOCK',
 'CDLBELTHOLD',
 'CDLBREAKAWAY',
 'CDLCLOSINGMARUBOZU',


In [32]:
talib.get_function_groups()

{'Cycle Indicators': ['HT_DCPERIOD',
  'HT_DCPHASE',
  'HT_PHASOR',
  'HT_SINE',
  'HT_TRENDMODE'],
 'Math Operators': ['ADD',
  'DIV',
  'MAX',
  'MAXINDEX',
  'MIN',
  'MININDEX',
  'MINMAX',
  'MINMAXINDEX',
  'MULT',
  'SUB',
  'SUM'],
 'Math Transform': ['ACOS',
  'ASIN',
  'ATAN',
  'CEIL',
  'COS',
  'COSH',
  'EXP',
  'FLOOR',
  'LN',
  'LOG10',
  'SIN',
  'SINH',
  'SQRT',
  'TAN',
  'TANH'],
 'Momentum Indicators': ['ADX',
  'ADXR',
  'APO',
  'AROON',
  'AROONOSC',
  'BOP',
  'CCI',
  'CMO',
  'DX',
  'MACD',
  'MACDEXT',
  'MACDFIX',
  'MFI',
  'MINUS_DI',
  'MINUS_DM',
  'MOM',
  'PLUS_DI',
  'PLUS_DM',
  'PPO',
  'ROC',
  'ROCP',
  'ROCR',
  'ROCR100',
  'RSI',
  'STOCH',
  'STOCHF',
  'STOCHRSI',
  'TRIX',
  'ULTOSC',
  'WILLR'],
 'Overlap Studies': ['BBANDS',
  'DEMA',
  'EMA',
  'HT_TRENDLINE',
  'KAMA',
  'MA',
  'MAMA',
  'MAVP',
  'MIDPOINT',
  'MIDPRICE',
  'SAR',
  'SAREXT',
  'SMA',
  'T3',
  'TEMA',
  'TRIMA',
  'WMA'],
 'Pattern Recognition': ['CDL2CROWS',
  'C

In [33]:

indicadores =indicadores_tecnicos('ABEV3.SA')
indicadores

[*********************100%***********************]  1 of 1 completed


,rsi,roc,willr
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
5581,43.558851,-6.255474,-71.449305
5582,43.558851,-6.255474,-71.449305
5583,43.558851,-6.255474,-71.449305
5584,43.558851,-6.255474,-71.449305


In [34]:
#ROC - Rate of change : ((price/prevPrice)-1)*100
roc = ROC(close, timeperiod=10)

NameError: name 'close' is not defined

In [ ]:
slowk, slowd = STOCH(high, low, close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
slowk
slowd

In [ ]:
fastk, fastd = STOCHRSI(close, timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)
fastk
fastd

In [ ]:
#Williams %R moves between zero and -100.above -20 is overbought.-80 is oversold.
willr = WILLR(high, low, close, timeperiod=14)
willr

In [ ]:
#  ATR serve para mensurar a volatilidade do mercado, porém sem indicar a direção
atr = ATR(high, low, close, timeperiod=14)

In [ ]:
# Moving Average Convergence/Divergence (MACD)
macd, macdsignal, macdhist = MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)

In [ ]:
macdhist

In [ ]:
bull_bear_power = CDLKICKINGBYLENGTH(open, high, low, close)
bull_bear_power

In [ ]:
# Engulfing Pattern
engulfing = CDLENGULFING(open, high, low, close)
engulfing

In [ ]:
 	technical_indicator 	value 	signal
RSI(14) 	52.2430 	neutral
1 	STOCH(9,6) 	82.0860 	overbought
2 	STOCHRSI(14) 	95.7880 	overbought
3 	MACD(12,26) 	-0.0500 	sell
4 	ADX(14) 	23.1400 	buy
5 	Williams %R 	-12.5000 	overbought
6 	CCI(14) 	82.6629 	buy
7 	ATR(14) 	0.4050 	less_volatility
8 	Highs/Lows(14) 	0.3014 	buy
9 	Ultimate Oscillator 	62.5370 	buy
10 	ROC 	6.6270 	buy
11 	Bull/Bear Power(13) 	0.0620 	buy

In [ ]:
#DYT = dividendo pago por ação *100 / preço da ação
df_diaria_final['DYT']= df_diaria_final.apply(lambda x: x.Dividends*100/ x.AMBEV if x.Dividends > 0 else 0, axis= 'columns')


In [ ]:
#sns.barplot(x= df_diaria['AMBEV'], y=df_diaria['DYT' ], data = df_diaria);
df_diaria['DYT'].plot();
df_diaria['AMBEV'].plot();


In [ ]:
import plotly.io as pio
import plotly.offline as py
import plotly.graph_objs as go
dados_grafico = [go.Scatter(x=df_diaria.index, y=df_diaria['AMBEV'])]
dados_div = [go.Scatter(x=df_diaria.index, y=df_diaria['DYT'])]
py.iplot(dados_grafico)


### Coletando as informações - para análise técnica e Fundamentalista

#### yfinance Ticker

In [ ]:
ticker.actions.head()

In [ ]:
ticker.calendar

In [ ]:
ticker.info

In [ ]:
ticker.major_holders 

In [ ]:
ticker.institutional_holders 

In [ ]:
ticker.news

#### Fundamentus

In [ ]:
url = 'https://www.fundamentus.com.br/resultado.php'
header = {

  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",

  "X-Requested-With": "XMLHttpRequest"

}

In [ ]:
r = requests.get(url, headers=header)
df = pd.read_html(r.text, decimal=',', thousands='.')[0]

df


In [ ]:
df[df.Papel == 'ABEV3']

##### Principais comandos yFinance Ticker

msft = yf.Ticker("MSFT")

# get stock info
msft.info

# get historical market data
hist = msft.history(period="max")

# show actions (dividends, splits)
msft.actions

# show dividends
msft.dividends

# show splits
msft.splits

# show financials
msft.financials
msft.quarterly_financials

# show major holders
msft.major_holders

# show institutional holders
msft.institutional_holders

# show balance sheet
msft.balance_sheet
msft.quarterly_balance_sheet

# show cashflow
msft.cashflow
msft.quarterly_cashflow

# show earnings
msft.earnings
msft.quarterly_earnings

# show sustainability
msft.sustainability

# show analysts recommendations
msft.recommendations

# show next event (earnings, etc)
msft.calendar

# show ISIN code - *experimental*
# ISIN = International Securities Identification Number
msft.isin

# show options expirations
msft.options

# show news
msft.news

#### Investpy -tecnico e fundamentalista

In [ ]:
#!pip install investpy

In [ ]:
dados_fundamentalista = inv.get_stock_information('ABEV3', country='brazil')
dados_fundamentalista

In [ ]:
indicadores_tecnicos_sinal = inv.technical_indicators('ABEV3', country='brazil', product_type='stock')
indicadores_tecnicos_sinal

In [ ]:
inv.moving_averages('ABEV3', country='brazil', product_type='stock')

### bcb currency 
- Alguns dados estavam faltando da cotação do dolar, só para constar acho que não será utilizado

In [ ]:
from bcb import currency

ao = currency.get(['USD'], start='2000-01-01', end=datetime.datetime.today() - datetime.timedelta(days=1))